In [ ]:
from itertools import product
import os

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import seaborn as sns

from eftqpe.physical_costing import (
    fermi_hubbard_walk_costs,
    fermi_hubbard_multicircuit_physical_cost,
    fermi_hubbard_ftqc_physical_cost,
)
from eftqpe.utils import make_decreasing_function

plt.style.use("figstyle.mplstyle")

## Logical costs of Fermi-Hubbard walk operator

In [ ]:
sides = np.arange(2, 20, dtype=int)
costs = np.array([fermi_hubbard_walk_costs(side) for side in sides])

fig, ax = plt.subplots(3, 1, figsize=(4, 6), sharex=True, gridspec_kw={'hspace': 0})

ax[0].plot(sides, costs[:,0], 'o-')
ax[0].set_ylabel('CCZ count')

ax[1].plot(sides, costs[:,1], 'o-', color='red')
ax[1].set_ylabel('Total qubits')

ax[2].plot(sides, costs[:,2], 'o-', color='green')
ax[2].set_ylabel(r'1-norm $\lambda$')

ax[-1].set_xlabel('Side length')

## Physical costs for circuit-division SinQPE

In [ ]:
save_file = 'data/physical_costs/fermi_hubbard_multicircuit_long_run.csv'

if os.path.exists(save_file):
    cost_df = pd.read_csv(save_file)
    delta_e = cost_df['delta_e'][0]
    side_list = np.sort(cost_df['side'].unique())
    n_factories_list = np.sort(cost_df['n_factories'].unique())
    gamma_list = np.sort(cost_df['gamma'].unique())
else:
    delta_e = 1e-2
    side_list = [2, 5, 10, 20]
    n_factories_list = [1, 2]
    # gamma_list = np.logspace(-5, -2, 10) # quicker test run
    gamma_list = np.logspace(-9, -2) # whole range

    costs = []

    for side, n_factories, gamma in tqdm(list(product(side_list, n_factories_list, gamma_list))):
        cost_dict = fermi_hubbard_multicircuit_physical_cost(
            side=side, gamma=gamma, delta_e=delta_e, n_factories=n_factories
        )
        # remove unnecessary keys
        cost_dict.pop("physical_cost", None)
        cost_dict.pop("factory", None)
        cost_dict.pop("data_block", None)
        # add input parameters
        input_params = {"side": side, "n_factories": n_factories, "gamma": gamma, "delta_e": delta_e}
        cost_dict = {**input_params, **cost_dict}
        costs.append(cost_dict)

    cost_df = pd.DataFrame(costs)

    os.makedirs(os.path.dirname(save_file), exist_ok=True)
    cost_df.to_csv(save_file, index=False)

In [ ]:
cost_df

In [ ]:
colors = sns.color_palette("flare", len(side_list))
fig, ax = plt.subplots(2, 2, figsize=(3.5, 3.5), sharex=True, sharey='row', gridspec_kw={'hspace': 0, 'wspace':0})

for k, n_factories in enumerate(n_factories_list):
    for j, side in enumerate(side_list):
        sub_df = cost_df.query(f'side == {side} & n_factories == {n_factories}')
        ax[0, k].plot(sub_df.gamma, sub_df.t_tot_hr, '-', color = colors[j])
        ax[0, k].plot(sub_df.gamma, sub_df.t_max_hr, '--', color = colors[j])
        ax[1, k].plot(sub_df.gamma, sub_df.footprint, '.', color = colors[j],
                      label=f'$L = {side}$')
        
    ax[0, k].set_title(f'{n_factories} CCZ factor'+('y' if n_factories == 1 else 'ies'))

ax[0,0].set_ylabel('runtime in hours')
ax[0,0].legend([plt.Line2D([0], [0], color='grey', linestyle=ls) for ls in ['-', '--']],
              [r'$\mathcal{T}_{\mathrm{tot}}$', r'$\mathcal{T}_{\mathrm{max}}$'], ncol=1)

ax[1,0].set_ylabel(r'\# physical qubits')
ax[1,1].legend(ncol=1)

fig.text(0.5, 0.05, r'$\gamma$, error rate per walk step', ha='center', va='top')

ax[0,0].set_yscale('log')
ax[1,0].set_yscale('log')
ax[1,0].set_xscale('log')

In [ ]:
fig = plt.figure(figsize=(3.5, 3.5))  # 4.1, 4.1
markers = ["+-", "x:"]

for i, n_factories in enumerate(n_factories_list):
    for j, side in enumerate(side_list):
        sub_df = cost_df.query(f"side == {side} & n_factories == {n_factories}")
        footprints, runtimes = make_decreasing_function(sub_df.footprint, sub_df.t_tot_hr)

        plt.plot(
            footprints,
            runtimes,
            markers[i],
            color=colors[j],
            label=f"$L = {side}$",
        )

handles = [
    plt.Line2D([0], [0], color="grey", marker=marker[0], linestyle=marker[1]) for marker in markers
] + [plt.Line2D([0], [0], color=c) for c in colors]

labels = [f"{n_factories} factories" for n_factories in n_factories_list] + [
    f"L = {side}" for side in side_list
]
plt.legend(handles, labels, ncol=2)

plt.axhline(24, color="black", linestyle="dashed")
plt.text(1.01e5, 1.3 * 24, "1 day", va="bottom")
plt.axhline(24 * 30, color="black", linestyle="dashed")
plt.text(1.5e5, 1.3 * 24 * 30, "1 month", va="bottom")

plt.xlabel(r"\# physical qubits")
plt.grid(axis="x", which="minor")
plt.ylabel("runtime in hours")
plt.yscale("log")
plt.xscale("log")

## Comparison to standard FTQC algorithm

In [ ]:
delta_e = 1e-2
side_list = [2, 5, 10, 20]
n_factories_list = [1, 2]
error_budget_list = [1e-2, 1e-3]

ftqc_costs = []

for side, n_factories, error_budget in tqdm(
    list(product(side_list, n_factories_list, error_budget_list))
):
    cost_dict = fermi_hubbard_ftqc_physical_cost(
        side=side, error_budget=error_budget, delta_e=delta_e, n_factories=n_factories,
    )
    # remove unnecessary keys
    cost_dict.pop("physical_cost", None)
    cost_dict.pop("factory", None)
    cost_dict.pop("data_block", None)
    # add input parameters
    cost_dict.update(
        {"side": side, "n_factories": n_factories, "error_budget": error_budget}
    )
    ftqc_costs.append(cost_dict)

ftqc_cost_df = pd.DataFrame(ftqc_costs)

In [ ]:
ftqc_cost_df

In [ ]:
fig = plt.figure(figsize=(3.4, 2.2))
colors = sns.color_palette("flare", len(side_list))
markers_ftqc = ["^", "v"]
n_factories = 1

for j, side in enumerate(side_list):
    sub_df = cost_df.query(f"side == {side} & n_factories == {n_factories}")
    footprints, runtimes = make_decreasing_function(sub_df.footprint, sub_df.t_tot_hr)

    plt.plot(
        footprints,
        runtimes,
        '.-',
        color=colors[j],
        label=f"$L = {side}$",
    )

    for i, error_budget in enumerate(error_budget_list):
        sub_df = ftqc_cost_df.query(f"side == {side} & n_factories == {n_factories} & error_budget == {error_budget}")
        plt.plot(
            sub_df.footprint,
            sub_df.runtime_hr,
            markers_ftqc[i],
            color=colors[j],
        )


handles = (
    [plt.Line2D([0], [0], color=c) for c in colors]
    +
    [
        plt.Line2D([0], [0], color="grey", marker=marker, linestyle="none")
        for marker in markers_ftqc
    ]
)
labels = (
    [f"L = {side}" for side in side_list]
    +
    [rf"{error_budget*100}\% fail" for error_budget in error_budget_list]
)
plt.legend(handles, labels, ncol=3, loc='lower center', bbox_to_anchor=(0.5, 1))

plt.axhline(24, color="black", linestyle="dashed")
plt.text(1.01e5, 1.3 * 24, "1 day", va="bottom")
plt.axhline(24 * 30, color="black", linestyle="dashed")
plt.text(1.5e5, 1.3 * 24 * 30, "1 month", va="bottom")

plt.xlabel(r"\# physical qubits")
plt.grid(axis="x", which="minor")
plt.ylabel("runtime in hours")
plt.yscale("log")
plt.xscale("log")